<a href="https://colab.research.google.com/drive/1XLHoR0ygN7JErx2DVwcOXAvRe_NjnlBk?usp=sharing" target="_blank">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/>
</a>

In [1]:
# 1. Setup and Imports
%load_ext tensorboard

import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import os
import datetime
from collections import OrderedDict

In [2]:
# 2. Load and Preprocess Fashion MNIST
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()

X_train_full = X_train_full.astype(np.float32) / 255.0
X_test = X_test.astype(np.float32) / 255.0

X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

# Flatten the data
X_train = X_train.reshape(-1, 28 * 28)
X_valid = X_valid.reshape(-1, 28 * 28)
X_test = X_test.reshape(-1, 28 * 28)


29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [3]:
# 3. Build a Model
model = keras.models.Sequential([
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])


In [4]:
# 4. Setup Optimizer, Loss, and Metrics
loss_fn = keras.losses.SparseCategoricalCrossentropy()
optimizer = keras.optimizers.Nadam(learning_rate=0.001)

train_loss_metric = keras.metrics.Mean()
train_acc_metric = keras.metrics.SparseCategoricalAccuracy()
val_loss_metric = keras.metrics.Mean()
val_acc_metric = keras.metrics.SparseCategoricalAccuracy()

In [5]:
# 5. Prepare Data Pipeline
batch_size = 32
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(10000).batch(batch_size)
valid_dataset = tf.data.Dataset.from_tensor_slices((X_valid, y_valid)).batch(batch_size)

In [6]:
# 6. TensorBoard Setup
logdir = os.path.join("logs", "custom_loop_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
writer = tf.summary.create_file_writer(logdir)


In [ ]:
# 7. Custom Training Loop
n_epochs = 10

for epoch in range(1, n_epochs + 1):
    print(f"Epoch {epoch}/{n_epochs}")

    for step, (X_batch, y_batch) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            y_pred = model(X_batch, training=True)
            loss = loss_fn(y_batch, y_pred)

        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        train_loss_metric.update_state(loss)
        train_acc_metric.update_state(y_batch, y_pred)

    train_loss = train_loss_metric.result().numpy()
    train_acc = train_acc_metric.result().numpy()

    # Validation
    for X_batch_val, y_batch_val in valid_dataset:
        y_val_pred = model(X_batch_val, training=False)
        val_loss_metric.update_state(loss_fn(y_batch_val, y_val_pred))
        val_acc_metric.update_state(y_batch_val, y_val_pred)

    val_loss = val_loss_metric.result().numpy()
    val_acc = val_acc_metric.result().numpy()

    # Log to TensorBoard
    with writer.as_default():
        tf.summary.scalar("Loss/Train", train_loss, step=epoch)
        tf.summary.scalar("Accuracy/Train", train_acc, step=epoch)
        tf.summary.scalar("Loss/Val", val_loss, step=epoch)
        tf.summary.scalar("Accuracy/Val", val_acc, step=epoch)

    print(f"  Train loss: {train_loss:.4f}, acc: {train_acc:.4f} | Val loss: {val_loss:.4f}, acc: {val_acc:.4f}")


# Reset metrics after each epoch
train_loss_metric.reset_state()
train_acc_metric.reset_state()
val_loss_metric.reset_state()
val_acc_metric.reset_state()


Epoch 1/10
  Train loss: 0.4388, acc: 0.8449 | Val loss: 0.3813, acc: 0.8641
Epoch 2/10
  Train loss: 0.4057, acc: 0.8557 | Val loss: 0.3643, acc: 0.8702
Epoch 3/10


In [ ]:
# 8. Launch TensorBoard
print("TensorBoard ready below:")
%tensorboard --logdir=logs --port=6018